In [2]:
import torch
from evotorch import Problem
from evotorch.algorithms import SteadyStateGA
from evotorch.operators import (
    SimulatedBinaryCrossOver,
    GaussianMutation,
)
from evotorch.logging import StdOutLogger

# Kursawe function with two conflicting objectives
def kursawe(x: torch.Tensor) -> torch.Tensor:
    f1 = torch.sum(
        -10 * torch.exp(
            -0.2 * torch.sqrt(x[:, 0:2] ** 2.0 + x[:, 1:3] ** 2.0)
        ),
        dim=-1,
    )
    f2 = torch.sum(
        (torch.abs(x) ** 0.8) + (5 * torch.sin(x ** 3)),
        dim=-1,
    )
    fitnesses = torch.stack([f1, f2], dim=-1)
    return fitnesses

prob = Problem(
    # Two objectives, both minimization
    ["min", "min"],
    kursawe,
    initial_bounds=(-5.0, 5.0),
    solution_length=3,
    vectorized=True,
)

# Works like NSGA-II for multiple objectives
ga = SteadyStateGA(prob, popsize=200)
ga.use(
    SimulatedBinaryCrossOver(
        prob,
        tournament_size=4,
        cross_over_rate=1.0,
        eta=8,
    )
)
ga.use(GaussianMutation(prob, stdev=0.03))
logger = StdOutLogger(ga)

ga.run(100)

[2023-04-03 16:09:14] INFO     < 2442> evotorch.core: Instance of `Problem` (id:140397249811120) -- The `dtype` for the problem's decision variables is set as torch.float32
[2023-04-03 16:09:14] INFO     < 2442> evotorch.core: Instance of `Problem` (id:140397249811120) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2023-04-03 16:09:14] INFO     < 2442> evotorch.core: Instance of `Problem` (id:140397249811120) -- The `device` of the problem is set as cpu
[2023-04-03 16:09:14] INFO     < 2442> evotorch.core: Instance of `Problem` (id:140397249811120) -- The number of actors that will be allocated for parallelized evaluation is 0


ValueError: Cannot compute the utility values, because there are solutions which are not evaluated yet.

In [6]:
import torch
from evotorch.algorithms import SNES
from evotorch.logging import StdOutLogger
from evotorch import Problem

# Minimize the Lennard-Jones atom cluster potential
def pairwise_distances(positions: torch.Tensor) -> torch.Tensor:
    positions = positions.view(positions.shape[0], -1, 3)
    deltas = positions.unsqueeze(2) - positions.unsqueeze(1)
    distances = torch.norm(deltas, dim=-1)
    print(positions.shape)
    return distances


def cluster_potential(positions: torch.Tensor) -> torch.Tensor:
    distances = pairwise_distances(positions)
    pairwise_cost = (1 / distances).pow(12) - (1 / distances).pow(6.0)
    ut_pairwise_cost = torch.triu(pairwise_cost, diagonal=1)
    potential = 4 * ut_pairwise_cost.sum(dim=(1, 2))
    return potential


problem = Problem(
    "min",
    cluster_potential,
    initial_bounds=(-1e-12, 1e-12),
    device="cuda:0" if torch.cuda.is_available() else "cpu",
    solution_length=150,
    # Evaluation is vectorized
    vectorized=True,
    # Higher-than-default precision
    dtype=torch.float64,
)

searcher = SNES(problem, popsize=1000, stdev_init=0.01)
logger = StdOutLogger(searcher, interval=100)

searcher.run(3)

[2023-04-03 16:11:55] INFO     < 2442> evotorch.core: Instance of `Problem` (id:140394023367632) -- The `dtype` for the problem's decision variables is set as torch.float64
[2023-04-03 16:11:55] INFO     < 2442> evotorch.core: Instance of `Problem` (id:140394023367632) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float64
[2023-04-03 16:11:55] INFO     < 2442> evotorch.core: Instance of `Problem` (id:140394023367632) -- The `device` of the problem is set as cpu
[2023-04-03 16:11:55] INFO     < 2442> evotorch.core: Instance of `Problem` (id:140394023367632) -- The number of actors that will be allocated for parallelized evaluation is 0
torch.Size([1000, 50, 3])
torch.Size([1000, 50, 3])
torch.Size([1000, 50, 3])
